In [22]:
%load_ext autoreload

%autoreload 2

import numpy as np
import networkx as nx
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1 - create a SBM graph and extract the edges list


In [31]:
size = 1000
s = size
sizes = [s, s, s]
probs = np.array([[0.45, 0.05, 0.02], [0.05, 0.45, 0.07], [0.02, 0.07, 0.40]])/10
g = nx.stochastic_block_model(sizes, probs, seed=0)
'''
pin = .05
pout = .1
probs = [[pin, pout, 0], [0.0, pin, pout], [pout, 0.0, pin]]
#probs = [[pin, pout, 0], [0.0, pin, pout], [pout, 0.0, pin]]

g = nx.stochastic_block_model(sizes, probs, seed=0, directed=True)
'''
num_nodes = len(g)
edges_list = []
for source, target in g.edges():
    edges_list.append([source, target])
edges_list = np.array(edges_list)


### node/edge colors and shape

In [37]:
# update edge colors
colors_by_block = [[1, 0, 0], [0, 1, 0,], [0, 0, 1]]
#colors_by_block = [[1, 0, 0, 240], [0, 1, 0, 240], [0, 0, 1, 0]]

edge_colors = []
for source, target in g.edges():
    c0 = colors_by_block[source//s]
    c1 = colors_by_block[target//s]
    edge_colors += [c0, c1]
    
colors_by_block = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
colors = np.array(
    [colors_by_block[i//s] 
     for i in range(len(g))]).astype('float64')


markers = [['o', 's', 'd'][i//s]  for i in range(len(g))]
edge_colors = np.array(edge_colors).astype('float64')

Exception in thread Thread-63671:
Exception in thread Thread-63670:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/devmessias/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
  File "/home/devmessias/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
        self.run()
  File "/home/devmessias/anaconda3/lib/python3.8/threading.py", line 1254, in run
self.run()
  File "/home/devmessias/anaconda3/lib/python3.8/threading.py", line 1254, in run
        self.function(*self.args, **self.kwargs)
  File "/home/devmessias/phd/fury/fury/stream/tools.py", line 855, in _run
self.function(*self.args, **self.kwargs)
    self.callback(*self.args, **self.kwargs)
  File "/home/devmessias/phd/fury/fury/stream/client.py", line 202, in interaction_callback
  File "/home/devmessias/phd/fury/fury/stream/tools.py", line 855, in _run
    ts = time.time()*1000
NameError: name 'time' is not defined
    self.callback(*self.args, **self.kw

### 2 - Creates a network superactor and adds to a FURY Scene

In [38]:
from helios import NetworkDraw
    
centers = np.random.normal(size=(len(g), 3))
test_uniforms = True
if test_uniforms:
    node_edge_opacity = .8
    node_edge_color = [255, 255, 255]
else:
    node_edge_opacity = np.random.uniform(size=len(g))
    node_edge_color = np.array(colors)
node_edge_width = node_edge_opacity
node_opacity = node_edge_opacity
network_draw = NetworkDraw(
        positions=centers,
        edges=edges_list,
        colors=colors,
        scales=1,
        node_edge_width=0,
        node_edge_opacity=node_edge_opacity,
        node_opacity=node_opacity,
        node_edge_color=node_edge_color,
        marker='3d',
        edge_line_color=edge_colors,
        edge_line_opacity=.8,
        edge_line_width=1,

)



In [39]:
#print(dir(network_super_actor.edges.vtk_actor.GetProperty()))

### 4- Starts the streaming

In [40]:
from fury.stream.widget import Widget
#widget = Widget(showm, encoding='webrtc', port=8777)
widget = Widget(network_draw.showm, ms_stream=16)
widget.start()
time.sleep(2)

16
url: http://localhost:7217?iframe=1&encoding=mjpeg


In [41]:
widget.return_iframe(300)

In [ ]:
network_draw.showm.scene.background((1, 1,1))
network_draw.refresh()

### 5 - Start Force-Directed async layout

- ms equals to 100 means that the vtk_positions will be updated at each 100 ms

In [29]:
from helios.layouts import HeliosFr
layout = HeliosFr(edges_list,network_draw, update_interval_workers=10)

In [30]:
layout.start(20)

In [10]:
layout.stop()

### 6 - Change some network properties

In [10]:
if not test_uniforms:
    network_draw.nodes.edge_width = np.ones(len(g))/2
    network_draw.nodes.edge_opacity = np.ones(len(g))/2
    network_draw.nodes.marker_opacity = np.random.uniform(size=len(g))
    network_draw.nodes.edge_color = np.ones((len(g), 3))
    network_draw.nodes.edge_color = colors - colors/2

else:
    network_draw.nodes.edge_width = 0
    network_draw.nodes.edge_opacity = .5
    network_draw.nodes.marker_opacity = .5
    network_draw.nodes.edge_color = [1, 1, 1]


network_draw.Render()


In [48]:
layout.stop()

In [49]:
widget.stop()